<a href="https://colab.research.google.com/github/Intruz01/COVID19_spread_prediction/blob/master/COVID19_spread_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## IMPORT LIBRARIES

import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt

In [0]:
## READ DATA FOR CASES, DEATH and RECOVERIES
# for this prediction I am using data posted and updated by CSSEGISandData on https://github.com/CSSEGISandData

df_confirmed_cases_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_recoveries_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [88]:
df_confirmed_cases_raw.sample(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20
26,NaN,Bolivia,-16.2902,-63.5887,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,2,3,10,10,11,11,12,12,15,19,24,27,29,32,43,61,74,81,97,107,115,123,132,139,157,183,194,210,264,268,275,300,330,354,397,441,465,493
102,NaN,Estonia,58.5953,25.0136,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,12,16,16,79,115,171,205,225,258,267,283,306,326,352,369,404,538,575,645,679,715,745,779,858,961,1039,1097,1108,1149,1185,1207,1258,1304,1309,1332,1373,1400,1434,1459,1512
224,NaN,Uruguay,-32.5228,-55.7658,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,4,8,29,50,79,94,110,158,162,162,189,217,238,274,304,310,338,338,350,369,400,400,406,424,424,456,473,494,480,480,483,492,502,502,508
104,NaN,Ethiopia,9.1450,40.4897,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,5,5,6,6,9,9,11,11,12,12,12,16,16,21,23,26,29,29,35,38,43,44,52,55,56,65,69,71,74,82,85,92,96,105
130,NaN,Iceland,64.9631,-19.0208,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,69,85,103,134,156,171,180,220,250,330,409,473,568,588,648,737,802,890,963,1020,1086,1135,1220,1319,1364,1417,1486,1562,1586,1616,1648,1675,1689,1701,1711,1720,1727,1739,1754,1760
176,NaN,Oman,21.0000,57.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,4,...,18,18,18,19,19,22,22,24,39,48,48,52,55,66,84,99,109,131,152,167,179,192,210,231,252,277,298,331,371,419,457,484,546,599,727,813,910,1019,1069,1180
126,NaN,Haiti,18.9712,-72.2852,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,2,2,6,7,8,8,8,8,15,15,15,16,16,18,20,21,24,25,27,30,31,33,33,40,40,41,41,43,44
162,NaN,Montenegro,42.5000,19.3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,3,14,14,21,27,47,52,69,82,84,85,91,109,123,144,174,201,214,233,241,248,252,255,263,272,274,283,288,303,303,307
185,NaN,Qatar,25.3548,51.1839,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,24,262,262,320,337,401,439,439,452,460,470,481,494,501,526,537,549,562,590,634,693,781,835,949,1075,1325,1604,1832,2057,2210,2376,2512,2728,2979,3231,3428,3711,4103,4663,5008
32,NaN,Cabo Verde,16.5388,-23.0418,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,4,4,5,5,6,6,6,6,6,6,7,7,7,7,7,7,7,8,8,10,11,56,56,56,58


In [89]:
# it's important to have data from yesterday, because the data from today are not complete yet
yesterday = datetime.date.today() - datetime.timedelta(days=1)
month = datetime.date.today().month


def check(dataset, thisdate=yesterday):
	url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{dataset}_global.csv'
	df = pd.read_csv(url, error_bad_lines=False)
	return df.loc[df['Country/Region'] == 'Poland'][thisdate.strftime('%m/%d/%y').lstrip('0').replace(' 0', ' ')].values[0]


print('Checking current data for day {} for Poland'.format(yesterday.strftime('%m/%d/%y')))
datasets_names_list = ['confirmed', 'deaths', 'recovered'] 
for _ in datasets_names_list:
	print('{}: {}'.format(_, check(_)))

Checking current data for day 04/18/20 for Poland
confirmed: 8742
deaths: 347
recovered: 981


In [0]:
def clean_and_melt_data(df):
  df_cleaned = df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_name='Cases', var_name='Date')
  return df_cleaned.set_index(['Country/Region', 'Province/State', 'Date'])


#clean all datasets
df_confirmed_cases = clean_and_melt_data(df_confirmed_cases_raw)
df_deaths = clean_and_melt_data(df_deaths_raw)
df_recoveries = clean_and_melt_data(df_recoveries_raw)